In [ ]:
!pip install transformers

In [ ]:
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import time
import datetime
import random
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import transformers
from transformers import AutoModel, BertTokenizerFast
from tqdm import tqdm
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
# specify GPU
device = torch.device("cuda")

In [ ]:
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os

os.chdir('/content/gdrive/My Drive/dataset/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_train = pd.read_json('all_train_small.json.gz', compression='gzip', lines=True)
#df_train = pd.read_json('all_train_medium.json.gz', compression='gzip', lines=True)
#df_train = pd.read_json('all_train_large.json.gz', compression='gzip', lines=True)
#df_train = pd.read_json('all_train_xlarge.json.gz', compression='gzip', lines=True)
valid = pd.read_csv('all_valid_small.csv')
#valid = pd.read_csv('all_valid_medium.csv')
#valid = pd.read_csv('all_valid_large.csv')
#valid = pd.read_csv('all_valid_xlarge.csv')
df_test = pd.read_json('task1_testset_1500_with_labels.json.gz', compression='gzip', lines=True)

In [ ]:
val = valid['pair_id'].tolist()
df_valid = df_train[df_train.pair_id.isin(val)]
df_valid.shape

(1808, 22)

In [ ]:
print('Original Shape of df_train : ', df_train.shape)
val = valid['pair_id'].tolist()
df_train = df_train[~df_train.pair_id.isin(val)]
print('Shape of df_train (after removal validate) : ', df_train.shape)
test = df_test['pair_id'].tolist()
df_train = df_train[~df_train.pair_id.isin(test)]
print('Shape of df_train (after removal test) : ', df_train.shape)

Original Shape of df_train :  (9038, 22)
Shape of df_train (after removal validate) :  (7230, 22)
Shape of df_train (after removal test) :  (7230, 22)


In [ ]:
df_train['description_left'] = df_train['description_left'].fillna('')
df_train['description_right'] = df_train['description_right'].fillna('')
df_train['left_temp'] = df_train['description_left']
#df_train['left_temp'] = df_train['title_left'] + df_train['description_left']
df_train['right_temp'] = df_train['description_right']
#df_train['right_temp'] = df_train['title_right'] + df_train['description_right']
df_train = df_train.replace(r'\n',' ', regex=True) 
df_train = df_train.replace(r'\\n',' ', regex=True)
df_train = df_train.replace(r'\t',' ', regex=True) 
df_train = df_train.loc[:, ['left_temp', 'right_temp', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']

In [ ]:
df_valid['description_left'] = df_valid['description_left'].fillna('')
df_valid['description_right'] = df_valid['description_right'].fillna('')
df_valid['left_temp'] = df_valid['description_left']
#df_valid['left_temp'] = df_valid['title_left'] + df_valid['description_left']
df_valid['right_temp'] = df_valid['description_right']
#df_valid['right_temp'] = df_valid'title_right'] + df_valid['description_right']
df_valid = df_valid.replace(r'\n',' ', regex=True) 
df_valid = df_valid.replace(r'\\n',' ', regex=True)
df_valid = df_valid.replace(r'\t',' ', regex=True) 
df_valid = df_valid.loc[:, ['left_temp', 'right_temp', 'label']]
df_valid.columns = ['text_a', 'text_b', 'label']

In [ ]:
df_test['description_left'] = df_test['description_left'].fillna('')
df_test['description_right'] = df_test['description_right'].fillna('')
df_test['left_temp'] = df_test['description_left']
#df_test['left_temp'] = df_test['title_left'] + df_test['description_left']
df_test['right_temp'] = df_test['description_right']
#df_test['right_temp'] = df_test['title_right'] + df_test['description_right']
df_test = df_test.replace(r'\n',' ', regex=True) 
df_test = df_test.replace(r'\\n',' ', regex=True)
df_test = df_test.replace(r'\t',' ', regex=True) 
df_test = df_test.loc[:, ['left_temp', 'right_temp', 'label']]
df_test.columns = ['text_a', 'text_b', 'label']

In [ ]:
#df_train = df_train[:15]

**Import BERT-base pretrained model**

In [ ]:
# import BERT-base pretrained model
#bert = AutoModel.from_pretrained('bert-base-uncased',return_dict=False)
bert = AutoModel.from_pretrained('bert-base-uncased')
#bert = AutoModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

#bert.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
'''
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig;
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased');

bert = BertForSequenceClassification.from_pretrained('bert-base-uncased')
'''

"\nfrom transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig;\ntokenizer = BertTokenizer.from_pretrained('bert-base-uncased');\n\nbert = BertForSequenceClassification.from_pretrained('bert-base-uncased')\n"

In [ ]:
# push the model to GPU
bert = bert.to(device)

In [ ]:
df_train.to_csv('train.tsv', sep='\t', index=False)
print("No. of Training Samples：", len(df_train))
df_train.head()

No. of Training Samples： 7230


,text_a,text_b,label
1,"""Description:StorageWorks Internal Tape Drive ...","""Description:HP SC40Ge Host Bus Adapter FIO P...",0
2,"""Description:Sun 73GB 1 10K FCAL Drive for Sto...","""Description:StorageWorks 300GB 10K Fibre Cha...",0
3,"""DDR4, 2666MHz, CL16, 1.2v, XMP 2.0, Lifetime ...",,1
4,"""Description:36.4GB Ultra320 3.5-inch SCSI Ho...",""" Long pro...",0
5,""" Artikel-Nr.: 3522 027/4700.00 ...",""" Rétro et épurée, cette montre automa...",0


In [ ]:
df_valid.to_csv('valid.tsv', sep='\t', index=False)
print("No. of Validate Samples：", len(df_valid))

No. of Validate Samples： 1808


In [ ]:
df_test.to_csv('test.tsv', sep='\t', index=False)
print("No. of Test Samples：", len(df_test))
df_test.head()

No. of Test Samples： 1500


,text_a,text_b,label
0,"""Intel's family of adapter, the Intel' Etherne...","""2 Port Intel E10G42BTDA 10 Gigabit SFP Ethern...",1
1,"""ZT-P10710B-10P, Core Clock: 1607MHz, Boost Cl...","""Zotac GeForce GTX 1070 Ti AMP! Extreme Editio...",1
2,"""TP-Link 54Mbps Pocket-Sized Wireless Print Se...","""Print servers give businesses the ability to ...",1
3,"""Logitech MK550 Wireless Wave Keyboard and Mou...","""The Logitech MK550 Wireless Wave Keyboard and...",1
4,,""" WD Blue PC SSD 500GB SATA III 2.5″ WDS500G1...",1


In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(bert.parameters(), lr = 2e-5) # lr 1e-3

# Begin

**Tokenize training, validation, test data by using BERT tokenizer**

In [ ]:
# encode vs encode plus vs batch encode plus

from torch.utils.data import Dataset

class ProductDataset(Dataset):

  def __init__(self, mode, tokenizer):
    assert mode in ['train', 'valid', 'test']
    self.mode = mode
    self.df = pd.read_csv(mode + '.tsv', sep='\t').fillna('')
    self.len = len(self.df)
    self.tokenizer = tokenizer  
    self.max_len = 512

  def __getitem__(self, idx):
    if self.mode == "test":
      text_a, text_b = self.df.iloc[idx, :2].values
      label_tensor = None
    else:
      text_a, text_b, label = self.df.iloc[idx, :].values
      label_tensor = torch.tensor(label)

    #encoded_pair = tokenizer.encode_plus(text_a, text_b, return_tensors='pt', add_special_tokens=True, max_length=self.max_len, truncation='longest_first')
    #tokens_tensor  = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
    #segments_tensor = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

    tok_seq1 = tokenizer.tokenize(text_a)
    tok_seq2 = tokenizer.tokenize(text_b)
    len1 = len(tok_seq1)
    len2 = len(tok_seq2)


    if len1 >= 254 and len2 < 254:
      length = 254 + 254 - len2
      if len1 >= length :
        tok_seq1 = tok_seq1[:length]
      #print(idx, ' case1', len(tok_seq1),len(tok_seq2))
    elif len1 < 254 and len2 >= 254:
      length = 254 + 254 - len1
      if len2 >= length :
        tok_seq2 = tok_seq2[:length]
      #print(idx, ' case2', len(tok_seq1),len(tok_seq2))
    else:
      tok_seq1 = tok_seq1[:254]
      tok_seq2 = tok_seq2[:254]
      #print(idx,' case3', len(tok_seq1),len(tok_seq2))

    input_ids = [tokenizer.cls_token_id]
    input_ids += tokenizer.convert_tokens_to_ids(tok_seq1)
    input_ids += [tokenizer.sep_token_id]
    token_type_ids = [0]*len(input_ids)
    input_ids += tokenizer.convert_tokens_to_ids(tok_seq2)
    input_ids += [tokenizer.sep_token_id]
    token_type_ids += [1]*(len(tok_seq2)+1) 
    #attention_mask = [1]*len(input_ids)

    #https://stackoverflow.com/questions/56360644/pytorch-runtimeerror-expected-tensor-for-argument-1-indices-to-have-scalar-t
    tokens_tensor  = torch.Tensor(input_ids).long()  # tensor of token ids
    segments_tensor = torch.Tensor(token_type_ids).long() # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens
    return (tokens_tensor, segments_tensor, label_tensor)
    

  def __len__(self):
    return self.len
    

trainset = ProductDataset('train', tokenizer=tokenizer)
validset = ProductDataset('valid', tokenizer=tokenizer)
testset = ProductDataset('test', tokenizer=tokenizer)

**Data Loader**

In [ ]:
# collate_fn , SequentialSampler

"""
BERT will return 4 tensors
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader,RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences

# 'samples' which pass to this function is a list，there are 3 tensors
# - tokens_tensor
# - segments_tensor
# - label_tensor
# zero padding will be applied on token tensor & segments tensor，and generate a masks tensors
def create_mini_batch(samples):
  tokens_tensors = [s[0] for s in samples]
  segments_tensors = [s[1] for s in samples]
    
  # handling when there are labels in dataset
  if samples[0][2] is not None:
    label_ids = torch.stack([s[2] for s in samples])
  else:
    label_ids = None
    
  # zero padding
  tokens_tensors = pad_sequence(tokens_tensors,batch_first=True)
  segments_tensors = pad_sequence(segments_tensors,batch_first=True)
  # attention masks will set the as 1 when the tokens_tensors is not zero, BERT will only pay attention to the non-zero tokens
  masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
  masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
  return tokens_tensors, masks_tensors, segments_tensors, label_ids


# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

#define a batch size
BATCH_SIZE = 8

# sampler for sampling the data during training
train_sampler = RandomSampler(trainset)

# sampler for sampling the data during training
val_sampler = SequentialSampler(validset)

train_dataloader = DataLoader(trainset, sampler=train_sampler,batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

val_dataloader = DataLoader(validset, sampler=val_sampler,batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

test_dataloader = DataLoader(testset,collate_fn=create_mini_batch)
#test_dataloader = DataLoader(testset, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

In [ ]:
def features_generation(dataloader):
  total_preds = []
  with torch.no_grad():  #https://clay-atlas.com/blog/2020/06/16/pytorch-cn-runtimeerror-cuda-out-of-memory/
    for step,batch in enumerate(dataloader):

      # push the batch to gpu
      batch = [r.to(device) for r in batch]
      sent_id, mask, segment, label = batch

      # clear previously calculated gradients 
      bert.zero_grad()

      # get model predictions for the current batch
      #outputs = bert(sent_id, mask, segment)
      #hidden_states = outputs[2]
      #word_embed = torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1)
      #word_embed = torch.stack(hidden_states[-4:]).sum(0)

      preds = bert(sent_id, mask, segment)
      features = preds[0][:,0,:].cpu().detach().numpy().tolist()
      #features = word_embed.cpu().detach().numpy().tolist()

      [total_preds.append(i) for i in features]
    return total_preds


In [ ]:
test_features = []
#word_embedding = []

with torch.no_grad():  #https://clay-atlas.com/blog/2020/06/16/pytorch-cn-runtimeerror-cuda-out-of-memory/
  for sent_id, segment, mask, label in test_dataloader:
    # get model predictions for the current batch
    sent_id, mask, segment = sent_id.to(device), mask.to(device), segment.to(device)

    preds = bert(sent_id, mask, segment)
    features = preds[0][:,0,:].cpu().detach().numpy().tolist()

    #outputs = bert(sent_id, mask, segment)
    #hidden_states = outputs[2]
    #features = [word_embedding.extend(hidden_states[i].cpu().detach().numpy().tolist()) for i in [-1,-2,-3,-4]]
    #features = word_embed.cpu().detach().numpy().tolist()

    [test_features.append(i) for i in features]
 

In [ ]:
train_features = features_generation(train_dataloader)
val_features = features_generation(val_dataloader)

In [ ]:
#https://stackoverflow.com/questions/17531796/find-the-dimensions-of-a-multidimensional-python-array
def dim(a):
    if not type(a) == list:
        return []
    return [len(a)] + dim(a[0])


print('df_train dimension: ', dim(train_features))
print('df_valid dimension: ', dim(val_features))
print('df_test dimension: ', dim(test_features))

df_train dimension:  [7230, 768]
df_valid dimension:  [1808, 768]
df_test dimension:  [1500, 768]


In [ ]:
def Xy_generation(l,df):
  temp_df = pd.DataFrame(l)
  df = df.reset_index(drop=True)
  df = pd.concat([df, temp_df], axis=1)
  y = df['label'].copy()
  X = df.drop(['text_a', 'text_b','label'], axis=1)
  return X,y

In [ ]:
X_train , y_train = Xy_generation(train_features,df_train)
X_val , y_val = Xy_generation(val_features,df_valid)
X_test , y_test = Xy_generation(test_features,df_test)

**Models**

In [ ]:
def get_confusion_matrix_values(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    return(cm[0][0], cm[0][1], cm[1][0], cm[1][1])

classifiers = {
    "DummyClassifier_stratified":DummyClassifier(strategy='stratified', random_state=0),    
    "KNeighborsClassifier":KNeighborsClassifier(3),
    "XGBClassifier":XGBClassifier(n_estimators=1000, learning_rate=0.1),
    "DecisionTreeClassifier":DecisionTreeClassifier(),
    "RandomForestClassifier":RandomForestClassifier(),
    "AdaBoostClassifier":AdaBoostClassifier(),
    "GradientBoostingClassifier":GradientBoostingClassifier(),
    "Perceptron": Perceptron(max_iter=40, eta0=0.1, random_state=1),
    "MLP": MLPClassifier(),
    "XGBClassifer tuned": XGBClassifier(colsample_bytree=0.8,
                      gamma=0.9,
                      max_depth=20,
                      min_child_weight=1,
                      scale_pos_weight=12,
                      subsample=0.9,
                      n_estimators=50, 
                      learning_rate=0.1)
}

df_results = pd.DataFrame(columns=['model', 'accuracy', 'mae', 'precision',
                                   'recall','f1','roc','run_time','tp','fp',
                                   'tn','fn'])

for key in classifiers:

    start_time = time.time()
    classifier = classifiers[key]
    model = classifier.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc = roc_auc_score(y_test, y_pred)
    classification = classification_report(y_test, y_pred, zero_division=0)
    run_time = format(round((time.time() - start_time)/60,2))
    tp, fp, fn, tn = get_confusion_matrix_values(y_test, y_pred)

    row = {'model': key,
           'accuracy': accuracy,
           'mae': mae,
           'precision': precision,
           'recall': recall,
           'f1': f1,
           'roc': roc,
           'run_time': run_time,
           'tp': tp,
           'fp': fp,
           'tn': tn,
           'fn': fn,
          }
    df_results = df_results.append(row, ignore_index=True)

df_results

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,model,accuracy,mae,precision,recall,f1,roc,run_time,tp,fp,tn,fn
0,DummyClassifier_stratified,0.562000,0.438000,0.330769,0.245714,0.281967,0.489011,0.0,714,261,129,396
1,KNeighborsClassifier,0.619333,0.380667,0.397321,0.169524,0.237650,0.515531,0.17,840,135,89,436
2,XGBClassifier,0.640667,0.359333,0.458333,0.146667,0.222222,0.526667,4.08,884,91,77,448
3,DecisionTreeClassifier,0.534667,0.465333,0.361156,0.428571,0.391986,0.510183,0.28,577,398,225,300
4,RandomForestClassifier,0.640000,0.360000,0.421053,0.076190,0.129032,0.509890,0.44,920,55,40,485
5,AdaBoostClassifier,0.645333,0.354667,0.474074,0.121905,0.193939,0.524542,0.47,904,71,64,461
6,GradientBoostingClassifier,0.648667,0.351333,0.493421,0.142857,0.221566,0.531941,2.44,898,77,75,450
7,Perceptron,0.528667,0.471333,0.375000,0.520000,0.435754,0.526667,0.0,520,455,273,252
8,MLP,0.589333,0.410667,0.409543,0.392381,0.400778,0.543883,0.52,678,297,206,319
9,XGBClassifer tuned,0.591333,0.408667,0.326772,0.158095,0.213094,0.491355,0.98,804,171,83,442
